In [ ]:
#import libraires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from tensorflow.keras import layers, callbacks
from tensorflow import keras
from imblearn.over_sampling import RandomOverSampler

In [ ]:
#read csv file
df = pd.read_csv('heart_2020.csv')

In [ ]:
#Check dataset
print("The Shape:",df.shape)
print("The NULL values:\n",df.isnull().sum())
print("Number of duplicated values",df.duplicated().sum())

In [ ]:
#drop duplicated rows
df.drop_duplicates(inplace=True)

In [ ]:
#descriptive statistics
df.describe()

In [ ]:
#summary of a DataFrame
df.info()

In [ ]:
#HeartDisease
plt.figure(figsize=(10,8), facecolor='azure')
plt.title('HeartDisease')
sns.countplot(x='HeartDisease', data=df)

In [ ]:
# select all categorical variables
df_categorical = df.select_dtypes(include=['object'])

In [ ]:
# apply Label encoder to df_categorical
enc= LabelEncoder()
df_categorical = df_categorical.apply(enc.fit_transform)
df.drop(df_categorical.columns, axis=1, inplace=True)
df = pd.concat([df_categorical, df], axis=1)


In [ ]:
# visualize correlations between all features
plt.figure(figsize=(12,8),facecolor='azure')
sns.heatmap(data=df.corr().round(2) , annot=True, cmap='coolwarm', cbar=False)
plt.title('Correlation Heatmap Between Numeric Variables',fontsize=18)
plt.tight_layout()
plt.show()

In [ ]:
feature=['Stroke', 'DiffWalking',
       'AgeCategory', 'Diabetic',
       'KidneyDisease','PhysicalHealth']

In [ ]:
# Define features and target variable
X=df.drop('HeartDisease', axis=1)
y=df['HeartDisease']

In [ ]:
# apply random oversampling to make dataste balace 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [ ]:
# Split data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42, stratify=y)

In [ ]:
# feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# build network model
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

In [ ]:
# apply early stopping 
early_stop = callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
# train the model 
history = model.fit(
    X_train, y_train,
    validation_split=.2,
    batch_size=256,
    epochs=100,
    callbacks=[early_stop],
)

In [ ]:
#compare the model loss against the actual val_loss
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot(title="Cross-Entropy Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

#compare the model accuracy against the actual val accuracy
history_df[['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()

In [ ]:
# Confusion matrix 
y_pred = model.predict(X_test_scaled)
y_pred_labels = (y_pred > 0.5).astype(int)

cm = metrics.confusion_matrix(y_test, y_pred_labels)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Calculate performance metrics
acc = metrics.accuracy_score(y_test, y_pred_labels)
prec = metrics.precision_score(y_test, y_pred_labels)
rec = metrics.recall_score(y_test, y_pred_labels)
f1 = metrics.f1_score(y_test, y_pred_labels)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")

In [ ]:
# test evaluate
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")